<a href="https://colab.research.google.com/github/martichgr/Algoritmos_Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_MG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Marta Gutiérrez Rubio<br>
Url: https://github.com/martichgr/Algoritmos_Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_MG.ipynb<br>
Google Colab: https://colab.research.google.com/drive/1qCkKF7NlpAkXvNExBQ1b1yRIyaVts7-u?usp=sharing <br>
Problema: Sesiones de doblaje

Descripción del problema:


Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:
* Número de actores: 10
* Número de tomas : 30
* Organización de tomas y actores: Se incluye en matriz doblaje                                  

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

#Modelo

###Espacio de soluciones
* Como entrada se tiene la matriz de doblaje, con 30 filas (tomas) y 10 columnas(actores). doblaje[i][j]==1 indica que el actor j actúa en la toma i, 0 en caso contrario.
* Por lo tanto, el espacio de soluciones es el número de maneras distintas de asignar las tomas en diferentes días, sin repetición, respetando un máximo de 6 tomas al día
* Como salida se tiene una lista de listas (mejor_plan), que representa cada uno de los días de la solución encontrada y para cada uno las tomas que se graban ese día. En el mejor de los casos, la longitud de mejor_plan será de 5 días, por la limitación de que se pueden grabar como máximo 6 tomas al días (30/6=5 días)


###Función objetivo
Minimizar el gasto del coste de actores de doblaje, es decir, la suma de actores distintos en cada día, de la lista mejor_plan.


###Restricciones
- Todos los actores cobran lo mismo y es por día de desplazamiento, independientemente del número de tomas que haga (1 unidad). Se crea un conjunto de actores por día, la longitud de éste será de ese dia
- No es posible grabar más de 6 tomas por día, por lo que el intervalo de días posibles es de 5 (solución óptima) a 30(peor solución). Se crea una variable para este valor.
- Cada toma debe grabarse en un día concreto

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

####Respuesta

El problema es un problema tipo de problemas de asignación/organización, que pertenecen a NP-duro, por lo que el número de soluciones hace que no se pueda resolver con algoritmos deterministas. Suponiendo el rodaje más corto (5 días) en el que cada día se cubren el máximo de tomas (supuesto que podría empeorarse si se supone que un día no llega a 6 tomas), aplicando las fórmulas de combinatoria, quedaría una complejidad de orden factorial: : 30! / (6!⁵)5!.

Por ello, se empleará uno de los algoritmos heurísticos vistos en clase. Concreamente, el de recocido simulado, debido a que su forma de proceder hace que se reduzca la complejidad.


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

####Respuesta
Como se ha comentado en el apartado anterior, se empleará la técnica de recocido simulado. Para este problema, debido a que las restricciones son de capacidad fija (6 tomas), el operador de "mover" o "intercambiar" las tomas garantiza que la resolución sea factible. Es un método adecuado para equilibrar la intensificación y la diversificación

In [2]:
import random
import math

# Máximo número de tomas que se pueden grabar en un mismo día
max_tomas_dia = 6

# Matriz de doblaje: filas = tomas, columnas = actores (1 si el actor interviene en la toma)
doblaje = [
    [1,1,1,1,1,0,0,0,0,0], [0,0,1,1,1,0,0,0,0,0], [0,1,0,0,1,0,1,0,0,0],
    [1,1,0,0,0,0,1,1,0,0], [0,1,0,1,0,0,0,1,0,0], [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,1,1,0,0,0,0,0], [1,1,0,0,0,1,0,0,0,0], [1,1,0,1,0,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,1,0], [1,1,1,0,1,0,0,1,0,0], [1,1,1,1,0,1,0,0,0,0],
    [1,0,0,1,1,0,0,0,0,0], [1,0,1,0,0,1,0,0,0,0], [1,1,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1], [1,0,1,0,0,0,0,0,0,0], [0,0,1,0,0,1,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0], [0,0,0,0,0,1,0,1,0,0],
    [1,1,1,1,0,0,0,0,0,0], [1,0,1,0,0,0,0,0,0,0], [0,0,1,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,1], [1,0,1,0,1,0,0,0,1,0], [0,0,0,1,1,0,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0], [1,0,0,0,1,1,0,0,0,0], [1,0,0,1,0,0,0,0,0,0]
]

# Fijamos la semilla para hacer reproducibles los experimentos y poder comparar resultados
random.seed(42)

# Creamos un conjunto de actores para cada toma
actores_toma = [set(i for i, v in enumerate(t) if v == 1) for t in doblaje]

#Función que devuelve el conjunto de actores que tienen que "moverse" ese día según las tomas de dicho día
def calcular_actores_dia(tomas_del_dia):
    if not tomas_del_dia:
        return set()
    res = set()
    for t in tomas_del_dia:
        # Unimos los actores de cada toma
        res.update(actores_toma[t])
    return res

#Función objetivo: sumatorio, para cada día, del número de actores que tiene doblaje dicho día
def obtener_coste_plan(plan):
    return sum(len(calcular_actores_dia(dia)) for dia in plan)


# Construimos una permutación aleatoria de las tomas
tomas_indices = list(range(len(doblaje)))
random.shuffle(tomas_indices)

# Se crea secuencialmente las tomas respetando max_tomas_dia
plan_actual = [tomas_indices[i:i + max_tomas_dia]
               for i in range(0, len(tomas_indices), max_tomas_dia)]
coste_actual = obtener_coste_plan(plan_actual)

# Inicializamos mejor solución conocida
mejor_plan = [dia[:] for dia in plan_actual]
mejor_coste = coste_actual

# Parámetros del recocido simulado
# temp: temperatura inicial
# iteraciones_por_temp: número de vecinos a explorar por nivel de temperatura
# tasa_enfriamiento: factor para ir reduciendo la temperatura, hasta llegar a 0.1
temp = 1000
iteraciones_por_temp = 100
tasa_enfriamiento = 0.995


while temp > 0.1:
    for _ in range(iteraciones_por_temp):
        # Elegimos dos días distintos al azar sobre los que aplicar el movimiento
        idx1, idx2 = random.sample(range(len(plan_actual)), 2)
        dia1, dia2 = plan_actual[idx1], plan_actual[idx2]

        # Coste de los dos días antes de hacer el cambio (para calcular la diferencia local)
        coste_previo_dias = len(calcular_actores_dia(dia1)) + len(calcular_actores_dia(dia2))

        # Movimiento candidato: o bien mover una toma de un día a otro, o intercambiar tomas
        tipo_mov = random.choice(['mover', 'intercambiar'])

        # Movimiento 1: mover una toma de dia1 a dia2
        if tipo_mov == 'mover' and len(dia1) > 0 and len(dia2) < max_tomas_dia:
            toma = random.choice(dia1)

            # Aplicamos el movimiento
            dia1.remove(toma)
            dia2.append(toma)

            # Recalculamos el coste local tras el cambio
            coste_nuevo_dias = len(calcular_actores_dia(dia1)) + len(calcular_actores_dia(dia2))
            diferencia = coste_nuevo_dias - coste_previo_dias

            # Criterio de aceptación de recocido simulado
            if diferencia < 0 or random.random() < math.exp(-diferencia / temp):
                coste_actual += diferencia
                # Si se encuentra un mejor solución se actualiza el coste y el plan seActualizamos mejor solución global si procede
                if coste_actual < mejor_coste:
                    mejor_coste = coste_actual
                    mejor_plan = [dias[:] for dias in plan_actual]
            else:
                # Deshacemos el movimiento si no conlleva a una mejor solución
                dia2.remove(toma)
                dia1.append(toma)

        # Movimiento 2: intercambiar una toma de dia1 con otra de dia2
        elif tipo_mov == 'intercambiar' and dia1 and dia2:
            i1, i2 = random.randrange(len(dia1)), random.randrange(len(dia2))

            # Guardamos las tomas para dejar claro qué se intercambia
            toma1, toma2 = dia1[i1], dia2[i2]

            # Intercambiamos
            dia1[i1], dia2[i2] = toma2, toma1

            coste_nuevo_dias = len(calcular_actores_dia(dia1)) + len(calcular_actores_dia(dia2))
            diferencia = coste_nuevo_dias - coste_previo_dias

            if diferencia < 0 or random.random() < math.exp(-diferencia / temp):
                coste_actual += diferencia
                if coste_actual < mejor_coste:
                    mejor_coste = coste_actual
                    mejor_plan = [d[:] for d in plan_actual]
            else:
                # Deshacemos el intercambio si no conlleva a una mejor solución
                dia1[i1], dia2[i2] = toma1, toma2

    # Enfriamos la temperatura después de explorar varios vecinos
    temp *= tasa_enfriamiento

# Se muestran los resultados
print(f"Valor función objetivo: {mejor_coste}")
print(f"Distribución plan de doblaje:")
for i, dia in enumerate(mejor_plan):
    if dia:
        # Sumamos uno porque los índices de tomas y actores comienzan en 0 y los ordenamos para que sea más legible al mostrarlos
        actores = sorted(actor + 1 for actor in calcular_actores_dia(dia))
        tomas = sorted(t + 1 for t in dia)
        print(f"\t - Día {i + 1} --> Tomas: {tomas} con los actores: {actores}")


Valor función objetivo: 27
Distribución plan de doblaje:
	 - Día 1 --> Tomas: [6, 7, 9, 16, 25, 28] con los actores: [1, 2, 4, 5, 10]
	 - Día 2 --> Tomas: [8, 10, 11, 21, 26, 29] con los actores: [1, 2, 3, 5, 6, 8, 9]
	 - Día 3 --> Tomas: [3, 4, 5, 13, 15, 27] con los actores: [1, 2, 4, 5, 7, 8]
	 - Día 4 --> Tomas: [1, 2, 12, 20, 22, 30] con los actores: [1, 2, 3, 4, 5, 6]
	 - Día 5 --> Tomas: [14, 17, 18, 19, 23, 24] con los actores: [1, 3, 6]
